# IMPORT LIBRARIES

In [11]:
import pandas as pd
import random
from pyomo.environ import *
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

In [12]:
# 1. Build the binary order–SKU matrix R
def build_R(df, order_col='Num. Ordine', sku_col='Articolo'):
    df_unique = df[[order_col, sku_col]].drop_duplicates()
    orders = df_unique[order_col].unique()
    skus = df_unique[sku_col].unique()
    order_idx = pd.Series(np.arange(len(orders)), index=orders)
    sku_idx = pd.Series(np.arange(len(skus)), index=skus)
    rows = df_unique[order_col].map(order_idx).values
    cols = df_unique[sku_col].map(sku_idx).values
    data = np.ones(len(df_unique), dtype=int)
    R = coo_matrix((data, (rows, cols)), shape=(len(orders), len(skus)))
    return R, orders, skus

In [13]:
# 3. GREEDY SEEDS WITH VOLUME heuristic
def greedy_seeds_volume(R, k_volume, sku_volumes, debug=False):
    """
    Greedy SEEDS allocation by DC seeds & coappearances, with volume constraints.
    """
    R_csr = R.tocsr()
    n_orders, n_skus = R_csr.shape
    n_dcs = len(k_volume)
    
    # Compute co-occurrence matrix and sales
    C = (R_csr.T).dot(R_csr).toarray()
    sales = np.diag(C)  # #orders per SKU
    
    # Sort SKUs by descending sales, DCs by descending capacity
    sku_by_sales = np.argsort(-sales)
    sorted_dcs    = np.argsort(-k_volume)
    
    if debug:
        print("Top 5 SKUs by sales:", sku_ids[sku_by_sales[:5]])
        print("DCs sorted by capacity:", sorted_dcs)
    
    allocation = [set() for _ in range(n_dcs)]
    remaining  = k_volume.copy()
    
    # 4. Seed largest DC with top SKU
    top_sku = sku_by_sales[0]
    top_dc  = sorted_dcs[0]
    if remaining[top_dc] >= sku_volumes[top_sku]:
        allocation[top_dc].add(top_sku)
        remaining[top_dc] -= sku_volumes[top_sku]
        if debug:
            print(f"Seed DC_{top_dc+1} with SKU {sku_ids[top_sku]}")
    
    # 5. Seed other DCs
    top_n = max(1, int(0.1 * n_skus))
    top_decile = set(sku_by_sales[:top_n])
    allocated_total = {top_sku}
    for d in sorted_dcs[1:]:
        # Candidates: unallocated & in top decile & fits capacity
        unallocated = [s for s in range(n_skus) if s not in allocated_total and sku_volumes[s] <= remaining[d]]
        cand = [s for s in unallocated if s in top_decile]
        if not cand:
            cand = unallocated
        # least coappearance with allocated_total
        scores = np.array([C[s, list(allocated_total)].sum() for s in cand])
        s_seed = cand[int(np.argmin(scores))]
        allocation[d].add(s_seed)
        remaining[d] -= sku_volumes[s_seed]
        allocated_total.add(s_seed)
        if debug:
            print(f"Seed DC_{d+1} with SKU {sku_ids[s_seed]} (least coappears)")

    # 6. Assign remaining SKUs by highest avg coappearance
    if debug:
        print("\nAssigning remaining SKUs by average coappearance")
    for s in sku_by_sales:
        if s in allocated_total: continue
        # for each DC with capacity
        best_dc, best_score = None, -np.inf
        for d in sorted_dcs:
            if sku_volumes[s] > remaining[d]: continue
            alloc_d = list(allocation[d])
            if alloc_d:
                avg = C[s, alloc_d].mean()
            else:
                avg = 0
            if avg > best_score:
                best_score, best_dc = avg, d
        if best_dc is not None:
            allocation[best_dc].add(s)
            remaining[best_dc] -= sku_volumes[s]
            allocated_total.add(s)
            if debug:
                print(f"SKU {sku_ids[s]} -> DC_{best_dc+1} (avg coappear {best_score:.1f})")

    # 7. Fill remaining capacity by highest coappearance
    if debug:
        print("\nFilling remaining capacity")
    while True:
        allocated_flag = False
        for d in sorted_dcs:
            cap = remaining[d]
            # candidates not allocated and fit
            cand = [s for s in range(n_skus) if s not in allocated_total and sku_volumes[s] <= cap]
            if not cand:
                continue
            scores = C[list(allocation[d]), :].sum(axis=0) if allocation[d] else C.sum(axis=0)
            mask = np.full(n_skus, -np.inf)
            mask[cand] = scores[cand]
            s_best = int(np.argmax(mask))
            allocation[d].add(s_best)
            remaining[d] -= sku_volumes[s_best]
            allocated_total.add(s_best)
            allocated_flag = True
            if debug:
                print(f"DC_{d+1}: added SKU {sku_ids[s_best]} (score {scores[s_best]:.0f})")
        if not allocated_flag:
            if debug:
                print("No further allocations possible.")
            break
    
    return allocation, remaining

# IMPORT DATA

In [14]:
# --------------------------
# Synthetic dataset generator
# --------------------------
np.random.seed(42)  # reproducibility

n_rows = 500

# Ranges / progressive labels
ordini = [f"ORD_{i}" for i in range(1, 1001)]
articoli = [f"ART_{i}" for i in range(1, 201)]
clienti = [f"CLI_{i}" for i in range(1, 101)]
percorsi = [f"PER_{i}" for i in range(1, 31)]
ecr1_vals = [f"ECR1_{i}" for i in range(1, 6)]
ecr2_vals = [f"ECR2_{i}" for i in range(1, 6)]
ecr3_vals = [f"ECR3_{i}" for i in range(1, 6)]
ecr4_vals = [f"ECR4_{i}" for i in range(1, 6)]

# Build DataFrame
df = pd.DataFrame({
    "Num. Ordine": np.random.choice(ordini, n_rows),
    "Mese-Giorno": pd.date_range("2025-01-01", periods=n_rows, freq="D").strftime("%m-%d"),
    "Articolo": np.random.choice(articoli, n_rows),
    "Pezzi evasi": np.random.randint(1, 12, n_rows),
    "Ecr1": np.random.choice(ecr1_vals, n_rows),
    "Ecr2": np.random.choice(ecr2_vals, n_rows),
    "Ecr3": np.random.choice(ecr3_vals, n_rows),
    "Ecr4": np.random.choice(ecr4_vals, n_rows),
    "Cliente": np.random.choice(clienti, n_rows),
    "Percorso": np.random.choice(percorsi, n_rows),
    "Giacenza Pezzi Volume [m3]": np.round(np.random.uniform(0.1, 10.0, n_rows), 6),
    "Volume evaso [m3]": np.round(np.random.uniform(0.001, 1.0, n_rows), 6),
})

print("✅ Synthetic dataset created:")
print(df.head())

✅ Synthetic dataset created:
  Num. Ordine Mese-Giorno Articolo  Pezzi evasi    Ecr1    Ecr2    Ecr3  \
0     ORD_103       01-01   ART_67            8  ECR1_3  ECR2_2  ECR3_3   
1     ORD_436       01-02  ART_104            5  ECR1_4  ECR2_4  ECR3_2   
2     ORD_861       01-03  ART_174            2  ECR1_2  ECR2_4  ECR3_4   
3     ORD_271       01-04   ART_24            6  ECR1_2  ECR2_5  ECR3_3   
4     ORD_107       01-05  ART_114            5  ECR1_5  ECR2_1  ECR3_3   

     Ecr4 Cliente Percorso  Giacenza Pezzi Volume [m3]  Volume evaso [m3]  
0  ECR4_3  CLI_75   PER_21                    7.067544           0.547690  
1  ECR4_1  CLI_64   PER_16                    3.795272           0.991781  
2  ECR4_4  CLI_81    PER_2                    5.680234           0.664538  
3  ECR4_1  CLI_51    PER_2                    4.561796           0.200047  
4  ECR4_5  CLI_53   PER_23                    6.753355           0.045916  


In [15]:
# --------------------------
# Extract df_stock_vol from df (using Giacenza Pezzi Volume from df)
# --------------------------

# articoli unici e media del volume pezzi già presenti nel df
df_stock_vol = (
    df.groupby("Articolo", as_index=False)["Giacenza Pezzi Volume [m3]"]
      .mean()
      .rename(columns={"Articolo": "ARTICOLO"})
)

# Assumiamo un volume pezzo random (es. da 0.05 a 3.5 m3)
df_stock_vol["Volume pezzo"] = np.round(np.random.uniform(0.05, 3.5, len(df_stock_vol)), 6)

# Calcoliamo la giacenza media = (Giacenza Pezzi Volume [m3] * 1000) / Volume pezzo
df_stock_vol["GIACENZA MEDIA"] = (
    df_stock_vol["Giacenza Pezzi Volume [m3]"] * 1000 / df_stock_vol["Volume pezzo"]
).round(1)

print("✅ df_stock_vol ricostruito da df:")
print(df_stock_vol.head())


✅ df_stock_vol ricostruito da df:
  ARTICOLO  Giacenza Pezzi Volume [m3]  Volume pezzo  GIACENZA MEDIA
0    ART_1                    5.855932      1.043018          5614.4
1   ART_10                    4.434571      0.371786         11927.8
2  ART_100                    2.629116      3.375190           779.0
3  ART_101                    6.327985      0.516027         12262.9
4  ART_102                    6.678560      0.573625         11642.7


# GREEDY ORDER

In [16]:
# 3. Execute the full pipeline

# Assume df and df_stock_vol are already loaded in the environment
R, order_ids, sku_ids = build_R(df)

# Compute SKU volumes (m3) from df_stock_vol
vol_series = df_stock_vol.set_index('ARTICOLO')['Giacenza Pezzi Volume [m3]']
median_vol = vol_series.median()
vol_series_filled = vol_series.fillna(median_vol)
sku_volumes = np.array([vol_series_filled.get(sku, median_vol) for sku in sku_ids], dtype=float)

# Define DC capacities (m3)
#capacity_A = 19800 * 0.37 #25000
#capacity_B =  12720 * 0.37 #6800

capacity_A = df.drop_duplicates(subset = 'Articolo').groupby('Ecr1')['Giacenza Pezzi Volume [m3]'].sum().sum()*0.65
capacity_B = df.drop_duplicates(subset = 'Articolo').groupby('Ecr1')['Giacenza Pezzi Volume [m3]'].sum().sum()*0.45

k_volume = np.array([capacity_A, capacity_B])

# Run heuristic with debug on
allocation, remaining_volume = greedy_seeds_volume(R, k_volume, sku_volumes, debug=True)
print("\nFinal remaining volumes (m3):", remaining_volume)

# 4. Map indices back to Articolo codes
alloc_map = []
for d, skus in enumerate(allocation):
    for s in skus:
        alloc_map.append({
            'Warehouse': f'DC_{d+1}', 
            'Articolo': sku_ids[s]
        })
df_allocation = pd.DataFrame(alloc_map)

Top 5 SKUs by sales: ['ART_144' 'ART_86' 'ART_145' 'ART_58' 'ART_99']
DCs sorted by capacity: [0 1]
Seed DC_1 with SKU ART_144
Seed DC_2 with SKU ART_86 (least coappears)

Assigning remaining SKUs by average coappearance
SKU ART_145 -> DC_1 (avg coappear 0.0)
SKU ART_58 -> DC_1 (avg coappear 0.0)
SKU ART_99 -> DC_1 (avg coappear 0.0)
SKU ART_161 -> DC_2 (avg coappear 1.0)
SKU ART_47 -> DC_1 (avg coappear 0.0)
SKU ART_117 -> DC_1 (avg coappear 0.2)
SKU ART_152 -> DC_1 (avg coappear 0.0)
SKU ART_118 -> DC_1 (avg coappear 0.0)
SKU ART_173 -> DC_1 (avg coappear 0.0)
SKU ART_69 -> DC_1 (avg coappear 0.0)
SKU ART_92 -> DC_1 (avg coappear 0.0)
SKU ART_191 -> DC_2 (avg coappear 0.5)
SKU ART_90 -> DC_1 (avg coappear 0.0)
SKU ART_186 -> DC_1 (avg coappear 0.0)
SKU ART_1 -> DC_2 (avg coappear 0.3)
SKU ART_39 -> DC_1 (avg coappear 0.0)
SKU ART_190 -> DC_1 (avg coappear 0.0)
SKU ART_141 -> DC_1 (avg coappear 0.0)
SKU ART_12 -> DC_2 (avg coappear 0.5)
SKU ART_101 -> DC_1 (avg coappear 0.1)
SKU ART_7

In [17]:
df_allocation

,Warehouse,Articolo
0,DC_1,ART_67
1,DC_1,ART_24
2,DC_1,ART_114
3,DC_1,ART_32
4,DC_1,ART_151
...,...,...
179,DC_2,ART_4
180,DC_2,ART_80
181,DC_2,ART_2
182,DC_2,ART_171


# STATISTICS

In [18]:
df = pd.merge(df, df_allocation, on='Articolo', how='left')
df['Warehouse'] = df['Warehouse'].replace({'DC_1': 'A', 'DC_2': 'B'})
df

,Num. Ordine,Mese-Giorno,Articolo,Pezzi evasi,Ecr1,Ecr2,Ecr3,Ecr4,Cliente,Percorso,Giacenza Pezzi Volume [m3],Volume evaso [m3],Warehouse
0,ORD_103,01-01,ART_67,8,ECR1_3,ECR2_2,ECR3_3,ECR4_3,CLI_75,PER_21,7.067544,0.547690,A
1,ORD_436,01-02,ART_104,5,ECR1_4,ECR2_4,ECR3_2,ECR4_1,CLI_64,PER_16,3.795272,0.991781,B
2,ORD_861,01-03,ART_174,2,ECR1_2,ECR2_4,ECR3_4,ECR4_4,CLI_81,PER_2,5.680234,0.664538,B
3,ORD_271,01-04,ART_24,6,ECR1_2,ECR2_5,ECR3_3,ECR4_1,CLI_51,PER_2,4.561796,0.200047,A
4,ORD_107,01-05,ART_114,5,ECR1_5,ECR2_1,ECR3_3,ECR4_5,CLI_53,PER_23,6.753355,0.045916,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,ORD_18,05-11,ART_184,9,ECR1_3,ECR2_4,ECR3_3,ECR4_2,CLI_29,PER_15,7.180346,0.781138,B
496,ORD_793,05-12,ART_145,7,ECR1_1,ECR2_2,ECR3_5,ECR4_4,CLI_59,PER_2,2.890999,0.791672,A
497,ORD_735,05-13,ART_74,9,ECR1_4,ECR2_5,ECR3_3,ECR4_3,CLI_98,PER_30,3.701715,0.602867,A
498,ORD_566,05-14,ART_17,6,ECR1_2,ECR2_4,ECR3_3,ECR4_1,CLI_17,PER_21,1.988187,0.512127,A


In [19]:
df_results = pd.DataFrame.from_dict(
    {
        'Magazzino A': [],
        'Magazzino B':[],
        
        'Codici in A':[],
        'Codici in B':[],
        
        'Stock [m3] in A':[],
        'Stock [m3] in B':[],
        
        '% Ordini completati in AB':[],
        '% Ordini completati in A':[], 
        '% Ordini completati in B':[],

        'Vol[m3] Ordini completati in A':[], 
        'Vol[m3] Ordini completati in B':[],
        'Vol[m3] Ordini completati in AB':[],
        'Vol[m3] Ordini completati in AB (A)':[],
        'Vol[m3] Ordini completati in AB (B)':[],
        
        '% Rotte completate in AB' :[],
        '% Rotte completate in A' :[],
        '% Rotte completate in B' :[],

        'Vol[m3] Rotte completati in A' :[],
        'Vol[m3] Rotte completati in B' :[],
        'Vol[m3] Rotte completati in AB' :[],
        'Vol[m3] Rotte completati in AB (A)' :[],
        'Vol[m3] Rotte completati in AB (B)' :[],

    }
)
df_results

,Magazzino A,Magazzino B,Codici in A,Codici in B,Stock [m3] in A,Stock [m3] in B,% Ordini completati in AB,% Ordini completati in A,% Ordini completati in B,Vol[m3] Ordini completati in A,...,Vol[m3] Ordini completati in AB (A),Vol[m3] Ordini completati in AB (B),% Rotte completate in AB,% Rotte completate in A,% Rotte completate in B,Vol[m3] Rotte completati in A,Vol[m3] Rotte completati in B,Vol[m3] Rotte completati in AB,Vol[m3] Rotte completati in AB (A),Vol[m3] Rotte completati in AB (B)


In [20]:
assignment_A = df.groupby('Warehouse')['Articolo'].unique().get('A', 0)
assignment_B = df.groupby('Warehouse')['Articolo'].unique().get('B', 0)

code_A = df.groupby('Warehouse')['Articolo'].nunique().get('A', 0)
code_B = df.groupby('Warehouse')['Articolo'].nunique().get('B', 0)
# print('Article division', article_division)

#Order analysis
order_grouped_df = df.groupby(['Mese-Giorno','Num. Ordine']).agg({
    'Warehouse': lambda x: list(x.unique()),  # Stores unique warehouses as lists
    'Volume evaso [m3]': 'sum'  # Sums up volume
}).reset_index()

order_movment_A = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(order_grouped_df) * 100
order_movment_B = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]) / len(order_grouped_df) * 100
order_movment_AB = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(order_grouped_df) * 100

order_vol_A = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum() 
order_vol_B = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]['Volume evaso [m3]'].sum() 
order_vol_AB = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum()

AB_order_list = list(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Num. Ordine'])
df_AB_order_volume = df.groupby(['Num. Ordine','Warehouse'])[['Volume evaso [m3]']].sum().reset_index()
df_AB_order_volume = df_AB_order_volume[df_AB_order_volume['Num. Ordine'].isin(AB_order_list)].groupby('Warehouse')['Volume evaso [m3]'].sum()

#Route analysis
route_grouped_df = df.groupby(['Mese-Giorno','Percorso']).agg({
    'Warehouse': lambda x: list(x.unique()),  # Stores unique warehouses as lists
    'Volume evaso [m3]': 'sum'  # Sums up volume
}).reset_index()

route_movment_A = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(route_grouped_df) * 100
route_movment_B = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]) / len(route_grouped_df) * 100
route_movment_AB = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(route_grouped_df) * 100

route_vol_A = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum() 
route_vol_B = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]['Volume evaso [m3]'].sum() 
route_vol_AB = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum()

# Step 1: Extract valid routes containing both 'A' and 'B' in 'Warehouse'
AB_order_list = route_grouped_df[
    route_grouped_df['Warehouse'].astype(str).str.contains(r'A') & 
    route_grouped_df['Warehouse'].astype(str).str.contains(r'B')
][['Mese-Giorno', 'Percorso']].apply(tuple, axis=1).tolist()

# Step 2: Compute total volume for each route
df_AB_route_volume = df.groupby(['Mese-Giorno', 'Percorso', 'Warehouse'])[['Volume evaso [m3]']].sum().reset_index()

# Step 3: Filter only the relevant routes and sum by Warehouse
df_AB_route_volume = df_AB_route_volume[
    df_AB_route_volume[['Mese-Giorno', 'Percorso']].apply(tuple, axis=1).isin(AB_order_list)
].groupby('Warehouse')['Volume evaso [m3]'].sum()


weighted_stock = df.groupby(['Articolo','Warehouse',])['Giacenza Pezzi Volume [m3]'].mean() * (df.groupby(['Articolo','Warehouse',])['Pezzi evasi'].sum() / df.groupby(['Articolo'])['Pezzi evasi'].sum())
stock_A = weighted_stock.groupby('Warehouse').sum().get('A', 0)
stock_B = weighted_stock.groupby('Warehouse').sum().get('B', 0)

df_results = pd.DataFrame.from_dict(
    {
        'Magazzino A': [assignment_A],
        'Magazzino B': [assignment_B],
        
        'Codici in A':[code_A],
        'Codici in B':[code_B],
        
        'Stock [m3] in A':[stock_A],
        'Stock [m3] in B':[stock_B],
        
        '% Ordini completati in AB':[order_movment_AB],
        '% Ordini completati in A':[order_movment_A],
        '% Ordini completati in B':[order_movment_B],

        '% Rotte completate in AB':[route_movment_AB],
        '% Rotte completate in A':[route_movment_A],
        '% Rotte completate in B':[route_movment_B],

        'Vol[m3] Ordini completati in A':[order_vol_A],
        'Vol[m3] Ordini completati in B':[order_vol_B],
        'Vol[m3] Ordini completati in AB':[order_vol_AB],
        'Vol[m3] Ordini completati in AB (A)':[df_AB_order_volume.get('A', 0)],
        'Vol[m3] Ordini completati in AB (B)':[df_AB_order_volume.get('B', 0)],


        'Vol[m3] Rotte completati in A':[route_vol_A],
        'Vol[m3] Rotte completati in B':[route_vol_B],
        'Vol[m3] Rotte completati in AB':[route_vol_AB],
        'Vol[m3] Rotte completati in AB (A)' :[df_AB_route_volume.get('A', 0)],
        'Vol[m3] Rotte completati in AB (B)' :[df_AB_route_volume.get('B', 0)],

        
    }
)

In [21]:
df_results

,Magazzino A,Magazzino B,Codici in A,Codici in B,Stock [m3] in A,Stock [m3] in B,% Ordini completati in AB,% Ordini completati in A,% Ordini completati in B,% Rotte completate in AB,...,Vol[m3] Ordini completati in A,Vol[m3] Ordini completati in B,Vol[m3] Ordini completati in AB,Vol[m3] Ordini completati in AB (A),Vol[m3] Ordini completati in AB (B),Vol[m3] Rotte completati in A,Vol[m3] Rotte completati in B,Vol[m3] Rotte completati in AB,Vol[m3] Rotte completati in AB (A),Vol[m3] Rotte completati in AB (B)
0,"[ART_67, ART_24, ART_114, ART_32, ART_151, ART...","[ART_104, ART_174, ART_175, ART_86, ART_161, A...",124,60,636.194027,330.533929,0.0,78.2,21.8,0.606061,...,186.255849,58.222264,0.0,0,0,184.621212,56.264707,3.592194,1.634637,1.957557
